<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/auto-keras/colorectal_histology/MobileNet_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install tensorflow-hub
%pip show tensorflow-hub
%pip show tensorflow

Name: tensorflow-hub
Version: 0.9.0
Summary: TensorFlow Hub is a library to foster the publication, discovery, and consumption of reusable parts of machine learning models.
Home-page: https://github.com/tensorflow/hub
Author: Google LLC
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, six, protobuf
Required-by: 
Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: astunparse, absl-py, grpcio, scipy, wrapt, numpy, wheel, h5py, google-pasta, gast, termcolor, keras-preprocessing, six, protobuf, tensorflow-estimator, tensorboard, opt-einsum
Required-by: fancyimpute


In [2]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

import tensorflow_datasets  as tfds
import tensorflow_hub as hub

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## データの読み込み

In [3]:
def fetch_tf_dataset(data_name, 
                     split=['train[0%:60%]','train[60%:80%]','train[80%:100%]'],
                     shuffle_files=True, as_supervised=True,
                     with_info=True, batch_size=None):
  '''TensorFlowデータセットからデータをフェッチ'''
  (ds_train, ds_val, ds_test), data_info = tfds.load(data_name, split=split,
                                  shuffle_files=shuffle_files,
                                  as_supervised=as_supervised,
                                  with_info=with_info,
                                  batch_size=batch_size)
  return ds_train, ds_val, ds_test, data_info
  


data_name = 'colorectal_histology' # @param{type:'string'}
x_name = 'image' # @param{type:'string'}
y_name = 'label' # @param{type:'string'}
# データのバッチサイズ
BATCH_SIZE = 256 # @param{type:'number'}

# データのフェッチ
ds_train, ds_val, ds_test, ds_info = fetch_tf_dataset(data_name=data_name,
                                     as_supervised=True,
                                     batch_size=BATCH_SIZE)

display(ds_info)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/colorectal_histology/2.0.0.incomplete70EC8B/colorectal_histology-train.tfrecord


Dataset colorectal_histology downloaded and prepared to /root/tensorflow_datasets/colorectal_histology/2.0.0. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='colorectal_histology',
    version=2.0.0,
    description='Classification of textures in colorectal cancer histology. Each example is a 150 x 150 x 3 RGB image of one of 8 classes.',
    homepage='https://zenodo.org/record/53169#.XGZemKwzbmG',
    features=FeaturesDict({
        'filename': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(150, 150, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=8),
    }),
    total_num_examples=5000,
    splits={
        'train': 5000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{kather2016multi,
      title={Multi-class texture analysis in colorectal cancer histology},
      author={Kather, Jakob Nikolas and Weis, Cleo-Aron and Bianconi, Francesco and Melchers, Susanne M and Schad, Lothar R and Gaiser, Timo and Marx, Alexander and Z{"o}llner, Frank Gerrit},
      journal={Scientific reports},
      volume={6},
      pages={27988},
      y

# モデル作成

## 前処理を行うKerasレイヤー
https://www.tensorflow.org/tutorials/images/data_augmentation?hl=ja

In [4]:
# リサイズ後の画像サイズ
RESIZE_IMG_SIZE = 224

# 画像のリサイズと、画素値のリスケールを行う層
resize_and_rescale = tf.keras.Sequential(layers=[
  layers.experimental.preprocessing.Resizing(height=RESIZE_IMG_SIZE,
                                             width=RESIZE_IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
], name='resize_and_rescale')


# Data Augmentationを行う層 
# 推論時には実行されない層
data_augmentation = tf.keras.Sequential(layers=[
                                          layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                          layers.experimental.preprocessing.RandomRotation(0.2),
                                        ], name='train_data_augmentation')

## ファインチューニングさせるMobileNet
前処理レイヤーの差し込みも行う

In [5]:
def _fetch_hub_keras_layer(hub_url, trainable=False):
  '''tensorflow hubのレイヤーをKerasレイヤーとしてfetch'''
  fetch_keras_layer = hub.KerasLayer(hub_url, trainable=trainable)
  return fetch_keras_layer


def clf_model_fn(hub_url, input_shape, output_shape, hub_layer_trainable=False):
  '''分類モデルの構造を定義'''
  model = Sequential([
                      resize_and_rescale, # 前処理用のレイヤー
                      data_augmentation, # 前処理用のレイヤー
                      _fetch_hub_keras_layer(hub_url, trainable=hub_layer_trainable),
                      layers.Dense(output_shape, activation='softmax')
  ])
  model.build(input_shape)
  return model


TF_HUB_URL = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4' #@param{type:'string'}
# 画像をリサイズする層があるためinputの画像サイズは未指定
MODEL_INPUT_SHAPE = (None, None, None, 3)
CLASS_NUM = ds_info.features[y_name].num_classes
FINE_TUNING = False

clf = clf_model_fn(hub_url=TF_HUB_URL, input_shape=MODEL_INPUT_SHAPE,
                   output_shape=CLASS_NUM, hub_layer_trainable=FINE_TUNING)
clf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resize_and_rescale (Sequenti (None, 224, 224, 3)       0         
_________________________________________________________________
train_data_augmentation (Seq (None, 224, 224, 3)       0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 8)                 10248     
Total params: 2,268,232
Trainable params: 10,248
Non-trainable params: 2,257,984
_________________________________________________________________


In [6]:
es = EarlyStopping(patience=3)

clf.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(0.001),
    metrics=['accuracy'],
)

In [7]:
EPOCHS = 100 #@param{type:'number'}

# バッチサイズはDatasetをfetchする際に指定済み
clf.fit(ds_train, epochs=EPOCHS,
        validation_data=ds_val,
        callbacks=[es])

Epoch 1/100
12/12 [==============================] - 6s 517ms/step - loss: 1.9872 - accuracy: 0.2727 - val_loss: 1.3924 - val_accuracy: 0.5290
Epoch 2/100
12/12 [==============================] - 4s 371ms/step - loss: 1.0405 - accuracy: 0.6403 - val_loss: 1.1308 - val_accuracy: 0.6490
Epoch 3/100
12/12 [==============================] - 4s 372ms/step - loss: 0.7428 - accuracy: 0.7630 - val_loss: 0.9466 - val_accuracy: 0.6980
Epoch 4/100
12/12 [==============================] - 4s 372ms/step - loss: 0.6314 - accuracy: 0.7963 - val_loss: 0.7990 - val_accuracy: 0.7350
Epoch 5/100
12/12 [==============================] - 4s 372ms/step - loss: 0.5733 - accuracy: 0.8080 - val_loss: 0.7346 - val_accuracy: 0.7560
Epoch 6/100
12/12 [==============================] - 4s 372ms/step - loss: 0.5261 - accuracy: 0.8297 - val_loss: 0.7001 - val_accuracy: 0.7700
Epoch 7/100
12/12 [==============================] - 5s 376ms/step - loss: 0.4987 - accuracy: 0.8363 - val_loss: 0.6800 - val_accuracy: 0.7740

# モデルの評価

### 簡易的な評価

In [8]:
%time
def evalute_model(model, ds):
  '''モデルの評価関数呼び出し'''
  score = model.evaluate(ds)
  return score


train_score = evalute_model(clf, ds_train)
val_score = evalute_model(clf, ds_val)
test_score = evalute_model(clf, ds_test)
print('訓練 loss:{loss}, 正答率:{auc}'.format(loss=train_score[0], auc=train_score[1]))
print('検証 loss:{loss}, 正答率:{auc}'.format(loss=val_score[0], auc=val_score[1]))
print('テスト loss:{loss}, 正答率:{auc}'.format(loss=test_score[0], auc=test_score[1]))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.63 µs
4/4 [==============================] - 1s 191ms/step - loss: 0.4052 - accuracy: 0.8580
訓練 loss:0.4026747941970825, 正答率:0.8606666922569275
検証 loss:0.485822856426239, 正答率:0.8450000286102295
テスト loss:0.4052494466304779, 正答率:0.8579999804496765
